In [23]:
from allennlp_models.pretrained import load_predictor

In [24]:
import checklist
from checklist.editor import Editor
from checklist.perturb import Perturb
from checklist.test_types import MFT, INV, DIR
from checklist.expect import Expect

In [25]:
from checklist.pred_wrapper import PredictorWrapper

In [26]:
import logging
logging.getLogger('allennlp.common.params').disabled = True 
logging.getLogger('allennlp.nn.initializers').disabled = True 
logging.getLogger('allennlp.modules.token_embedders.embedding').disabled = True 
logging.getLogger('urllib3.connectionpool').disabled = True 
logging.getLogger('allennlp.common.plugins').disabled = True 
logging.getLogger('allennlp.common.model_card').disabled = True 
logging.getLogger('allennlp.models.archival').disabled = True 
logging.getLogger('allennlp.data.vocabulary').disabled = True 
logging.getLogger('cached_path').disabled = True

In [27]:
srl_predictor = load_predictor('structured-prediction-srl')
output = srl_predictor.predict("John met a friend and smiled.")
output

{'verbs': [{'verb': 'met',
   'description': '[ARG0: John] [V: met] [ARG1: a friend] and smiled .',
   'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'I-ARG1', 'O', 'O', 'O']},
  {'verb': 'smiled',
   'description': '[ARG0: John] met a friend and [V: smiled] .',
   'tags': ['B-ARG0', 'O', 'O', 'O', 'O', 'B-V', 'O']}],
 'words': ['John', 'met', 'a', 'friend', 'and', 'smiled', '.']}

In [28]:
def predict_srl(data):
    
    pred = []
    for d in data:
        pred.append(srl_predictor.predict(d))
    return pred

predict_and_conf = PredictorWrapper.wrap_predict(predict_srl)

In [29]:
d = ["John met a friend and smiled."]
pred = predict_and_conf(d)
pred

([{'verbs': [{'verb': 'met',
     'description': '[ARG0: John] [V: met] [ARG1: a friend] and smiled .',
     'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'I-ARG1', 'O', 'O', 'O']},
    {'verb': 'smiled',
     'description': '[ARG0: John] met a friend and [V: smiled] .',
     'tags': ['B-ARG0', 'O', 'O', 'O', 'O', 'B-V', 'O']}],
   'words': ['John', 'met', 'a', 'friend', 'and', 'smiled', '.']}],
 array([1.]))

In [30]:
def format_srl(x, pred, conf, label=None, meta=None):
    
    return pred['verbs'][0]['description']

In [31]:
def get_arg(pred, arg_target='ARG1'):
    # we assume one predicate:
    predicate_arguments = pred['verbs'][0]
    #print(predicate_arguments)
    words = pred['words']
    tags = predicate_arguments['tags']
    
    arg_list = []
    for t, w in zip(tags, words):
        arg = t
        if '-' in t:
            arg = t.split('-')[1]
        if arg == arg_target:
            arg_list.append(w)
    arg_set = set(arg_list)
    return arg_set

In [32]:
# e = get_arg(pred,arg_target='ARG1')
# print(e)

In [33]:
def format_srl(x, pred, conf, label=None, meta=None):
    results = []
    predicate_structure = pred['verbs'][0]['description']
        
    return predicate_structure

In [34]:
def found_arg1_people(x, pred, conf, label=None, meta=None):
    
    # people should be recognized as arg1

    people = set([meta['first_name'], meta['last_name']])
    arg_0 = get_arg(pred, arg_target='ARG1')

    if arg_0 == people:
        pass_ = True
    else:
        pass_ = False
    return pass_


expect_arg0 = Expect.single(found_arg1_people)

In [41]:
# initialize editor object
editor = Editor()

# create examples
t = editor.template("John met {first_name} {last_name} and smiled.", meta=True,nsamples=1000, remove_duplicates=True)

#print(type(t))

for k, v in t.items():
    print(k, v)

meta [{'first_name': 'Benjamin', 'last_name': 'Hart'}, {'first_name': 'Jean', 'last_name': 'Moore'}, {'first_name': 'Karen', 'last_name': 'Foster'}, {'first_name': 'Richard', 'last_name': 'Bell'}, {'first_name': 'Sally', 'last_name': 'Morgan'}, {'first_name': 'Lisa', 'last_name': 'King'}, {'first_name': 'Ian', 'last_name': 'Hamilton'}, {'first_name': 'Tom', 'last_name': 'Martin'}, {'first_name': 'William', 'last_name': 'Crawford'}, {'first_name': 'Larry', 'last_name': 'Clarke'}, {'first_name': 'Frederick', 'last_name': 'Lewis'}, {'first_name': 'Evelyn', 'last_name': 'Hamilton'}, {'first_name': 'Anthony', 'last_name': 'Baker'}, {'first_name': 'Carl', 'last_name': 'Johnson'}, {'first_name': 'Lauren', 'last_name': 'Hill'}, {'first_name': 'Betty', 'last_name': 'Turner'}, {'first_name': 'Rachel', 'last_name': 'Jones'}, {'first_name': 'Lucy', 'last_name': 'Rose'}, {'first_name': 'Annie', 'last_name': 'Coleman'}, {'first_name': 'Rebecca', 'last_name': 'Miller'}, {'first_name': 'Alan', 'last_n

In [42]:
# initialize a test object
test = MFT(**t, name = 'detect_arg0_name_default_position', expect=expect_arg0)
test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)

Predicting 1000 examples
Test cases:      1000
Fails (rate):    22 (2.2%)

Example fails:
[ARG0: John] [V: met] [ARG1: Edwin] [ARGM-TMP: Foster] and smiled .
----
[ARG0: John] [V: met] [ARGM-TMP: Maria Foster] and smiled .
----
[ARG0: John] [V: met] [ARG1: Chris] [ARGM-TMP: Henderson] and smiled .
----


In [37]:
for k, v in test.results.items():
    print(k, v)

preds [{'verbs': [{'verb': 'met', 'description': '[ARG0: John] [V: met] [ARG1: Emma Taylor] and smiled .', 'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'I-ARG1', 'O', 'O', 'O']}, {'verb': 'smiled', 'description': '[ARG0: John] met Emma Taylor and [V: smiled] .', 'tags': ['B-ARG0', 'O', 'O', 'O', 'O', 'B-V', 'O']}], 'words': ['John', 'met', 'Emma', 'Taylor', 'and', 'smiled', '.']}, {'verbs': [{'verb': 'met', 'description': '[ARG0: John] [V: met] [ARG1: Kenneth Reed] and smiled .', 'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'I-ARG1', 'O', 'O', 'O']}, {'verb': 'smiled', 'description': '[ARG0: John] met Kenneth Reed and [V: smiled] .', 'tags': ['B-ARG0', 'O', 'O', 'O', 'O', 'B-V', 'O']}], 'words': ['John', 'met', 'Kenneth', 'Reed', 'and', 'smiled', '.']}, {'verbs': [{'verb': 'met', 'description': '[ARG0: John] [V: met] [ARG1: Eric Hall] and smiled .', 'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'I-ARG1', 'O', 'O', 'O']}, {'verb': 'smiled', 'description': '[ARG0: John] met Eric Hall and [V: smiled] .', 'tags': ['B

In [43]:
import sys
#[https://howtodoinjava.com/examples/python-print-to-file/]
# Saving the reference of the standard output
original_stdout = sys.stdout  
 
with open('result items argument ellipsis.txt', 'w') as f:
    sys.stdout = f 
    for k, v in test.results.items():
        print(k, v, file = f)
    # Reset the standard output
    sys.stdout = original_stdout 

In [ ]:
## will save to a json file at some point, code not ready yet: 

In [44]:
results_items = {}
for k, v in test.results.items():
    results_items = {k, v}


import json 

with open('result items argument ellipsis.json', 'w') as f:
    json_item = json.dumps(results_items)
    f.write(json_item)
    

TypeError: unhashable type: 'list'